In [2]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np

In [3]:
## Acessando pelo pacote da BD

df=bd.read_sql("SELECT ano, tema, sigla_uf,policia_militar_efetivo_total FROM `basedosdados-projetos.republica.indicadores_estadic_gerais` where ano=2019 and tema='seguranca_publica' order by sigla_uf", billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)

Downloading: 100%|██████████| 27/27 [00:00<00:00, 66.19rows/s]


In [6]:
df

,ano,tema,sigla_uf,policia_militar_efetivo_total
0,2019,seguranca_publica,AC,2338
1,2019,seguranca_publica,AL,6561
2,2019,seguranca_publica,AM,8746
3,2019,seguranca_publica,AP,3402
4,2019,seguranca_publica,BA,31783
5,2019,seguranca_publica,CE,21022
6,2019,seguranca_publica,DF,10896
7,2019,seguranca_publica,ES,8710
8,2019,seguranca_publica,GO,13575
9,2019,seguranca_publica,MA,11011


In [ ]:
## Utilizando como fonte para população a projeção populacional do IBGE em 2019 (data da pesquisa)

In [4]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Documentos auxiliares')

In [5]:
pop = pd.read_csv("pop_projetada_2019_ibge.csv", sep=";")

In [22]:
pop

,nome_uf,sigla_uf,pop. Projetada
0,Rondônia,RO,1777225
1,Acre,AC,881935
2,Amazonas,AM,4144597
3,Roraima,RR,605761
4,Pará,PA,8602865
5,Amapá,AP,845731
6,Tocantins,TO,1572866
7,Maranhão,MA,7075181
8,Piauí,PI,3273227
9,Ceará,CE,9132078


In [6]:
pop['pop. Projetada']= pop['pop. Projetada'].str.replace('.','')

In [7]:
pop['pop. Projetada']= pd.to_numeric(pop['pop. Projetada'])

In [21]:
pop.columns

Index(['ano', 'nome_uf', 'sigla_uf', 'pop_projetada'], dtype='object')

In [10]:
pop['ano']=2019

In [12]:
pop=pop[['ano','nome_uf', 'sigla_uf', 'pop. Projetada']]

In [20]:
pop=pop.rename(columns={'pop. Projetada':'pop_projetada'})

In [8]:
## Subindo para datalake
client = bigquery.Client()
dataset_ref = client.dataset('Datalake')


In [22]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de implementação da legislação.'),
 bigquery.SchemaField('nome_uf','STRING',description='Nome da UF.'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('pop_projetada','INTEGER',description='Sigla da Unidade da Federação.')
 ]

In [23]:
table_ref = dataset_ref.table('IBGE_projecao_populacional')

In [24]:
job_config = bigquery.LoadJobConfig(schema=schema)

In [25]:
job = client.load_table_from_dataframe(pop, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=6fb963d8-30f6-4245-bce3-43adb8086a0a>

In [31]:
### Join com o dado de policial

df1 = df.merge(pop[['sigla_uf', 'pop_projetada']],right_on='sigla_uf', left_on='sigla_uf')

In [38]:
df1

,ano,tema,sigla_uf,policia_militar_efetivo_total,pop_projetada,taxa
0,2019,seguranca_publica,AC,2338,881935,2.650989
1,2019,seguranca_publica,AL,6561,3337357,1.965927
2,2019,seguranca_publica,AM,8746,4144597,2.110217
3,2019,seguranca_publica,AP,3402,845731,4.022556
4,2019,seguranca_publica,BA,31783,14873064,2.13695
5,2019,seguranca_publica,CE,21022,9132078,2.301995
6,2019,seguranca_publica,DF,10896,3015268,3.613609
7,2019,seguranca_publica,ES,8710,4018650,2.167395
8,2019,seguranca_publica,GO,13575,7018354,1.934214
9,2019,seguranca_publica,MA,11011,7075181,1.556285


In [37]:
df1['taxa'] = df1['policia_militar_efetivo_total']/df1['pop_projetada']*1000

Subindo o arquivo para o gbq

In [39]:
## Subindo para datalake
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')


In [46]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ano                            27 non-null     Int64  
 1   tema                           27 non-null     object 
 2   sigla_uf                       27 non-null     object 
 3   policia_militar_efetivo_total  27 non-null     Int64  
 4   pop_projetada                  27 non-null     int64  
 5   taxa                           27 non-null     Float64
dtypes: Float64(1), Int64(2), int64(1), object(2)
memory usage: 1.6+ KB


In [51]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de implementação da legislação.'),
 bigquery.SchemaField('tema','STRING',description='Tema de origem daquele dado na ESTADIC'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('policia_militar_efetivo_total','INTEGER',description='Quantitativo total de efetivo de policiais militares.'),
 bigquery.SchemaField('pop_projetada','INTEGER',description='População projetada pelo IBGE para o ano de referência'),
 bigquery.SchemaField('taxa','FLOAT',description='taxa de policiais por 1000 habitantes')
 
 ]

In [53]:
table_ref = dataset_ref.table('ESTADIC_policial_militar_habitantes')

In [54]:
job_config = bigquery.LoadJobConfig(schema=schema)

In [55]:
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=93997f4c-dffd-4e73-b717-56bb99693853>